<a href="https://colab.research.google.com/github/tkhan3/textanalytics/blob/master/gensim_topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
#import pandas as pd
!google-drive-ocamlfuse drive
!apt-get -qq install -y graphviz && pip install -q pydot
!pip3 install seaborn==0.9.0
import pydot
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install sklearn_pandas

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131322 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import os
from os import chdir,listdir,path,getcwd
from sys import argv,exit
import pandas as pd
import re
#from mkl import *
import numpy as np
import pickle
import datetime
import timeit
from collections import Counter
from copy import deepcopy
from math import floor,ceil
from scipy.stats import gaussian_kde
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_samples,silhouette_score
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples,silhouette_score
import string

In [0]:
from keras.models import Model
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout, Embedding, LSTM, MaxPooling1D, Flatten
from keras.layers import Convolution1D, AveragePooling1D, Conv1D, Input, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Nadam, RMSprop
from keras.utils.np_utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [0]:
from keras.models import Model
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout, Embedding, LSTM, MaxPooling1D, Flatten
from keras.layers import Convolution1D, AveragePooling1D, Conv1D, Input, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Nadam, RMSprop
from keras.utils.np_utils import to_categorical
from keras.models import load_model

In [0]:
from scipy.stats import gaussian_kde
from statsmodels.nonparametric.kde import KDEUnivariate
from sklearn.manifold import TSNE
import colorsys

In [0]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import matplotlib.gridspec as gridspec
import nltk

In [0]:
!pip install pyLDAvis

    100% |████████████████████████████████| 1.6MB 12.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [0]:
%matplotlib inline

In [0]:
import gensim

In [0]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [0]:
def clean_up(rec):
  table = str.maketrans({key: None for key in string.punctuation})
  rec = str(rec)
  rec = rec.replace('\\n','').replace('\\r','')
  rec = re.sub(r'[^a-zA-Z0-9\.,;?<>/ ]',r'',rec)
  #rec = re.sub(r'\w*\d\w*', '', rec)
  rec = rec.strip()
  rec = re.sub(' +', ' ',rec)
  rec = rec.translate(table)
  rec = rec.lower()
  return rec

In [0]:
import os
drive_path = 'drive/DL_COLAB/'

TICKET_ANALYTICS_PATH = os.path.join(drive_path,"text_analytics","ticket_analysis")

def load_ticket_data(filename, file_path=TICKET_ANALYTICS_PATH):
    csv_path = os.path.join(file_path, filename)
    return pd.read_csv(csv_path,sep=",")

In [0]:
ticket_data_full = load_ticket_data("Telco_Apr-2016.csv",TICKET_ANALYTICS_PATH)

In [0]:
FIELDS = ["Defect ID","Title","Problem Details","Resolution Details","Root Cause"]

In [0]:
ticket_data = ticket_data_full[FIELDS]

In [0]:
problem_desc_list = ticket_data[['Problem Details']].values.tolist()

In [0]:
problem_desc_cleanned = [clean_up(item) for item in problem_desc_list]

In [0]:
problem_desc_cleanned[1:10]

['reason and sub reason section not updated under auc004 and auc005 in dbrd v007',
 'no functional requirements for auc003 dealer changes number of bulk receivers ba will add frs related to auc003',
 'no functional requirements for auc005 dealer places upgrade order ba added frs covering auc005',
 'should be automatically defaulting to all included when it is just going to general market right now',
 'auc covering migration of properties to bulk is missing there are two functional requirements present but no auc ba listed them under deployment option after several discussions ba was convinced to add auc for that purpose',
 'no auc nor functional requirements separation for fulfillment and nonfulfillment dealer activations ba agreed to add auc for nonfulfillment and fulfillment dealer receiver activation dws is uded to activate nonfulfillment fstp riocc and ivr are used for fulfillment dealer activations',
 'confirmation letter codes not updated in cor v200',
 'when attempting to upgrad

In [0]:
problem_desc_cleanned_df = pd.DataFrame(problem_desc_cleanned,columns=['Problem Details'])

In [0]:
#complete_text = ''.join(problem_desc_cleanned)

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
#words = nltk.tokenize.word_tokenize(complete_text)
words_list = list()
for rec in problem_desc_cleanned:
  words = nltk.tokenize.word_tokenize(rec)
  words_list.append(words)
#for rec in problem_desc_cleanned
#words_list = nltk.tokenize.word_tokenize(problem_desc_cleanned)

In [0]:
words_list[:1]

[['step',
  '3',
  'is',
  'the',
  'same',
  'between',
  'auc002',
  'and',
  'auc003',
  'it',
  'has',
  'to',
  'be',
  'eliminated',
  'from',
  'auc003',
  'nba',
  'agreed',
  'to',
  'remove']]

In [0]:
#from nltk.corpus import stopwords
#stop = set(stopwords.words('english'))
#clean_word = [word for word in words if word not in stop]

In [0]:
bigram = gensim.models.Phrases(words_list, min_count=4, threshold=100) 

In [0]:
trigram = gensim.models.Phrases(bigram[words_list], threshold=100)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [0]:
# Remove Stop Words
data_words_nostops = remove_stopwords(words_list)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['step', 'auc', 'eliminate', 'nba', 'agree', 'remove']]


In [0]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [0]:
id2word[0]

'agree'

In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [0]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.045*"offer" + 0.032*"intent" + 0.031*"call" + 0.027*"group" + '
  '0.023*"click" + 0.022*"display" + 0.021*"rio" + 0.020*"result" + '
  '0.018*"accept" + 0.017*"section"'),
 (1,
  '0.033*"error" + 0.021*"message" + 0.019*"activation" + 0.018*"issue" + '
  '0.018*"datum" + 0.015*"new" + 0.015*"order" + 0.015*"result" + '
  '0.015*"activate" + 0.014*"step"'),
 (2,
  '0.032*"request" + 0.025*"response" + 0.020*"success" + 0.018*"service" + '
  '0.016*"get" + 0.015*"name" + 0.013*"value" + 0.012*"call" + 0.012*"rank" + '
  '0.012*"code"'),
 (3,
  '0.049*"would" + 0.024*"datum" + 0.022*"account" + 0.018*"test" + '
  '0.014*"shopping_cart" + 0.013*"session" + 0.013*"offer" + '
  '0.012*"application" + 0.012*"net" + 0.010*"launch"'),
 (4,
  '0.044*"account" + 0.041*"order" + 0.039*"genie" + 0.026*"offer" + '
  '0.021*"select" + 0.017*"customer" + 0.017*"mini" + 0.017*"click" + '
  '0.013*"test" + 0.013*"upgrade"')]


In [0]:
topics_matrix = lda_model.show_topics(formatted=False, num_words=20)

([('offer', 0.045314483),
  ('intent', 0.032199714),
  ('call', 0.030853383),
  ('group', 0.027038556),
  ('click', 0.023086326),
  ('display', 0.021538464),
  ('rio', 0.021144511),
  ('result', 0.019715236),
  ('accept', 0.017857285),
  ('section', 0.017460903),
  ('search', 0.01637668),
  ('button', 0.015389647),
  ('select', 0.014575066),
  ('expect', 0.013863151),
  ('account', 0.0135541335),
  ('system', 0.012338369),
  ('proposition', 0.012251184),
  ('step', 0.011797346),
  ('actual', 0.010145933),
  ('member', 0.009994031)],)

In [0]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      31.376616        1       1 -0.172347  0.058813
0      26.662701        1       2 -0.068128 -0.204132
1      17.915934        1       3 -0.071004  0.189707
3      13.582479        1       4 -0.007208 -0.073168
2      10.462269        1       5  0.318688  0.028780, topic_info=     Category        Freq              Term       Total  loglift  logprob
term                                                                     
392   Default  526.000000             genie  526.000000  30.0000  30.0000
23    Default  676.000000             order  676.000000  29.0000  29.0000
84    Default  442.000000             would  442.000000  28.0000  28.0000
533   Default  384.000000            intent  384.000000  27.0000  27.0000
70    Default  233.000000           request  233.000000  26.0000  26.0000
63    Default  323.000000             error  323.000000  25.0000  25.0000
93    Default  331.000000             group  331.000000  24.0000  24.0000
67    Default  958.000000             offer  958.000000  23.0000  23.0000
109   Default  224.000000          response  224.000000  22.0000  22.0000
60    Default  908.000000           account  908.000000  21.0000  21.0000
335   Default  489.000000              call  489.000000  20.0000  20.0000
222   Default  340.000000             datum  340.000000  19.0000  19.0000
159   Default  206.000000            accept  206.000000  18.0000  18.0000
575   Default  228.000000              mini  228.000000  17.0000  17.0000
46    Default  147.000000        activation  147.000000  16.0000  16.0000
140   Default  168.000000           message  168.000000  15.0000  15.0000
282   Default  239.000000           service  239.000000  14.0000  14.0000
237   Default   89.000000           success   89.000000  13.0000  13.0000
25    Default  177.000000           upgrade  177.000000  12.0000  12.0000
45    Default  114.000000          activate  114.000000  11.0000  11.0000
8     Default  272.000000           section  272.000000  10.0000  10.0000
619   Default  142.000000       proposition  142.000000   9.0000   9.0000
96    Default  199.000000             issue  199.000000   8.0000   8.0000
876   Default  140.000000           session  140.000000   7.0000   7.0000
179   Default  365.000000               rio  365.000000   6.0000   6.0000
420   Default  118.000000              name  118.000000   5.0000   5.0000
354   Default  208.000000             exist  208.000000   4.0000   4.0000
186   Default  152.000000               get  152.000000   3.0000   3.0000
654   Default  173.000000     shopping_cart  173.000000   2.0000   2.0000
320   Default  116.000000            member  116.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
2043   Topic5   20.864328  appropriate_bulk   21.582844   2.2235  -5.3805
1981   Topic5   20.256306          httpwwww   20.973490   2.2226  -5.4100
155    Topic5   19.791967           project   20.514708   2.2215  -5.4332
632    Topic5   19.381725             match   20.137295   2.2192  -5.4542
2637   Topic5   17.957771               rel   18.675827   2.2182  -5.5305
1616   Topic5   17.358234        originator   18.075184   2.2169  -5.5644
1296   Topic5   17.225069       bulkfreetvs   17.943331   2.2165  -5.5721
1679   Topic5   17.263718         datastate   17.984003   2.2165  -5.5699
789    Topic5   16.994049   eligible_action   17.714745   2.2159  -5.5856
2085   Topic5   16.919716        retryallow   17.640179   2.2157  -5.5900
496    Topic5   27.027489          category   28.757469   2.1954  -5.1217
1301   Topic5   24.571995              tldb   26.657066   2.1759  -5.2169
70     Topic5  146.415985           request  233.024567   1.7927  -3.4321
109    Topic5  112.277824          response  224.032181   1.5666  -3.6975
420    Topic5   66.727005              name  118.606163   1.6822  -4.2179
111    Topic5   51.497433  